# Comparing the performance of Neural networks and Gradient boosted trees  , and a stacked ensemble of both in MHC class I binding prediction


---
### Background

The prediction of MHC peptide binding is an important method in immunology. Data driven methods give best performance for this task because of complexity of the MHC molecules and the binding process. The most successful machine learning models were Neural networks in this area.  I have seen articles describing SVM models, but nobody has tried gradient boosted trees (as far as I know).


In the last years the vast majority of Data science competitions were won with a framework called Gradient Boosted Trees. These models very frequently outperform neural networks in gereal machine learning tasks. 

These facts gave me the idea to compare the perfomance of  Neural networks  and Gradient boosted trees and in MHC class I peptide binding prediction.

Also Neural networks and Gradient boosted trees are very different models, therefore ensembling them can give a really big boost to performance. So I tried to ensemble these models.


----
### Data

#### Source
I have downloaded MHC class I peptide binding data which has been used in this benchmark article ( http://www.ncbi.nlm.nih.gov/pubmed/25017736)  from the IEDB website.


#### Encoding 

Input data is very simply coded as the folowing:
- zero padded sequences to the longest peptide length
- amino acids are encoded in a one hot scheme
- species are encoded in a one hot scheme
- HLA types are encoded in a one-hot scheme

#### Target 

I predicted binary binding labels based on the IC > 500nM threshold.


---

### Models

#### Neural network

I used a neural network architecture similar to the one used in NetMHCpan.
- 1 hidden layer
- 60 units in the hidden layer

I used the Keras software library. http://keras.io


#### Gradient boosted trees
Some resources about the model:
- http://xgboost.readthedocs.org/en/latest/model.html
- http://homes.cs.washington.edu/~tqchen/pdf/BoostedTree.pdf
    
I used the XGBoost software library. http://xgboost.readthedocs.org/en/latest/


#### Ensembling



I used a stacking approach for ensembling. I used original features, and probabilities predicted by base models as input for the meta estimator. The same folds are used in both step to prevent label leakage.

Original article about stacking.
- http://www.sciencedirect.com/science/article/pii/S0893608005800231




---

### Methods


I made 5-fold cross validation evaluations, with AUC for all the measurements.



--- 

### Conclusions


Gradient boosted trees significantly outperform neural networks in this task. The ensemble of neural networks and gradient boosted trees gives significant improvement over the base models. NetMHCPan outperforms my simple models.



Method | 5 fold CV
--- | --- 
NN | 
Gradient boosted trees | 
Ensemble of Gradient boosted trees + NN | 0.9362 
NetMHCpan | 0.937


---


### Future Work

#### Training on similar data than NetMHCpan

My models perform worse than NetMHCpan which is considered to be the state of the art model in MHC class I prediction. 

But NetMHCpan uses sophsticated encogind of the MHC molecules, a sophicsticated way for predicting different length peptides, adds random not binding data to inputs, and it is an ensemble of models. I think that Gradient Boosted trees trained on the data used for NetMHCpan training, and using ensembling could significanlty improve the current state of the art.



----
#### Notes

- To run this notebook you need to 
    - download the dataset
    - export the MHC_DATA variable
    - and have keras,theno,xgboost,sklearn,pandas,numpy installed.
    
    
- This notebook can be run as it is. Just click run all. 
    - It takes some time but on a stronger computer it should be less than 30 minutes.


----


This notebook was created by Dezso Ribli

### Load modules

In [23]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelBinarizer
from sklearn.cross_validation import KFold
from sklearn.cross_validation import train_test_split
import xgboost as xgb
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import roc_auc_score
import glob

#go to working dir
import os
os.chdir(os.environ['MHC_DIR'])

### Data Loader functions

In [2]:
def load_data():
    """Load train/blind test data as in the benchmark."""
    
    #load the cross validation data
    dataf='benchmark_mhci_reliability/binding/bd2009.1/bdata.2009.mhci.public.1.txt'
    data=pd.read_csv(dataf,sep='\t')
     
    #encode hla,species,amino acids,in a one hot scheme
    X_hla=one_hot_encode(data.mhc.values,data.mhc.values)
    X_species=one_hot_encode(data.species.values,data.species.values)
    X_seq=encode_seq(data.sequence.values,data.sequence.values)
    
    #stack columns
    X=np.column_stack([X_species,X_hla,X_seq,data.peptide_length.values])
    
    #make binding class labels
    y=np.ones(len(data.meas))
    y[data.meas.values>=500]=0

    #permute arrays as they are ordered!!!
    rng=np.random.RandomState(42)
    perm=rng.permutation(len(X))
    X,y=X[perm],y[perm]
    
    return X,y


def one_hot_encode(x_all,x_in):
    """One hot encode categorical variable."""
    lb = LabelBinarizer()
    lb.fit(x_all)
    x_out=lb.transform( x_in )
    return x_out


def encode_seq(x_all,x_in):
    """Encode string amino acid sequences to numbers."""
    #zero pad peptids to equal length
    maxlen=np.max(map(len,x_all))
    x_temp_all=np.array([list(x.zfill(maxlen)) for x in x_all])
    x_temp_in=np.array([list(x.zfill(maxlen)) for x in x_in])
    
    lb = LabelBinarizer()
    lb.fit(x_temp_all.flatten())
    x_out=np.column_stack([lb.transform(x_temp_in[:,i]) for i in range(maxlen)])

    return x_out

----


### Cross validation scheme for both models


In [12]:
def cv(fitter,model_spec,X,y,n_folds=5):
    """Evaluate XGB model with cross validation."""
    #result
    y_pred=np.zeros(len(y))
    #loop folds
    for train_index,test_index in KFold(len(X),n_folds=n_folds):
        #train and predict
        y_pred[test_index]=fitter(model_spec,X[train_index],y[train_index],X[test_index])
    return y_pred


def keras_fit_predict(get_model,X_train,y_train,X_test,
                      validation_split=0.1,patience=1,nb_epoch=100,**kwargs):
    """Fit model on train test set with early stopping."""
    #get model
    model=get_model(X_train.shape[1])
    
    #callbacks
    best_model=ModelCheckpoint('best_model',save_best_only=True,verbose=2)
    early_stop=EarlyStopping(patience=patience,verbose=2)
    
    #train it
    model.fit(X_train,y_train,nb_epoch = nb_epoch,validation_split=validation_split,
              callbacks=[best_model,early_stop],verbose=2,**kwargs)
    
    #load best model and predict
    model.load_weights('best_model')
    y_pred_test=model.predict(X_test).ravel()
    
    return y_pred_test

def xgb_fit_predict(params,X_train,y_train,X_test,num_boost_round=5000,verbose_eval=500,
                    early_stopping_rounds=200,validation_size=0.1):
    """Fit model on train test set with early stopping."""
    #split validation data for early stopping
    X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=validation_size)
    
    #convert to data format for xgb
    dtrain = xgb.DMatrix( X_train, label=y_train)
    dvalid = xgb.DMatrix( X_valid, label=y_valid)
    dtest = xgb.DMatrix( X_test )
    
    #define printed evaluations
    evallist  = [(dtrain,'train'),(dvalid,'eval')]

    #lets train
    bst = xgb.train(params,dtrain,evals=evallist,
                    num_boost_round=num_boost_round,
                    early_stopping_rounds=early_stopping_rounds,
                    verbose_eval=verbose_eval)
    
    #make predictions
    y_pred_test=bst.predict(dtest)
    
    return y_pred_test

### Load data

In [4]:
X,y=load_data()

### Train NN in a CV scheme

In [ ]:
#create a very simple NN model
def get_model(input_dim,n_dense=1024):
    """Creates Keras model."""
    model = Sequential()
    model.add(Dense(n_dense, input_dim=input_dim,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model

nn_cv_pred=cv(keras_fit_predict,get_model,X,y)

Train on 99110 samples, validate on 11013 samples
Epoch 1/100
Epoch 00000: val_loss improved from inf to 0.37779, saving model to best_model
46s - loss: 0.4517 - val_loss: 0.3778
Epoch 2/100
Epoch 00001: val_loss improved from 0.37779 to 0.31539, saving model to best_model
59s - loss: 0.3344 - val_loss: 0.3154
Epoch 3/100
Epoch 00002: val_loss improved from 0.31539 to 0.31414, saving model to best_model
66s - loss: 0.2848 - val_loss: 0.3141
Epoch 4/100
Epoch 00003: val_loss improved from 0.31414 to 0.29760, saving model to best_model
69s - loss: 0.2521 - val_loss: 0.2976
Epoch 5/100
Epoch 00004: val_loss did not improve
73s - loss: 0.2261 - val_loss: 0.3009
Epoch 6/100
Epoch 00005: val_loss did not improve
Epoch 00005: early stopping
73s - loss: 0.2033 - val_loss: 0.3061
Train on 99110 samples, validate on 11013 samples
Epoch 1/100
Epoch 00000: val_loss improved from inf to 0.36627, saving model to best_model
49s - loss: 0.4510 - val_loss: 0.3663
Epoch 2/100
Epoch 00001: val_loss impro

### Train gradient boosted trees in a CV scheme

In [17]:
#model params
params = {'max_depth':20,
         'eta':0.1,
         'min_child_weight':5,
         'colsample_bytree':1,
         'subsample':1,
         'silent':1,
         'objective': "binary:logistic",
         'eval_metric': 'auc',
         'nthread':4}

xgb_cv_pred=cv(xgb_fit_predict,params,X,y)

Will train until eval error hasn't decreased in 200 rounds.
[0]	train-auc:0.779680	eval-auc:0.752137
[500]	train-auc:0.986674	eval-auc:0.926433
[1000]	train-auc:0.994557	eval-auc:0.929153
[1500]	train-auc:0.997449	eval-auc:0.929363
Stopping. Best iteration:
[1355]	train-auc:0.996846	eval-auc:0.929613

Will train until eval error hasn't decreased in 200 rounds.
[0]	train-auc:0.784438	eval-auc:0.766076
[500]	train-auc:0.986822	eval-auc:0.928112
[1000]	train-auc:0.994702	eval-auc:0.929339
Stopping. Best iteration:
[1266]	train-auc:0.996545	eval-auc:0.929783

Will train until eval error hasn't decreased in 200 rounds.
[0]	train-auc:0.779405	eval-auc:0.763856
[500]	train-auc:0.987058	eval-auc:0.931594
[1000]	train-auc:0.994787	eval-auc:0.932908
Stopping. Best iteration:
[844]	train-auc:0.993201	eval-auc:0.933077

Will train until eval error hasn't decreased in 200 rounds.
[0]	train-auc:0.781964	eval-auc:0.763041
[500]	train-auc:0.987058	eval-auc:0.926184
[1000]	train-auc:0.994780	eval-auc:0

### Create a stacked ensemble of my NN and boosted tree models

In [ ]:
X_stacked=np.column_stack([X,xgb_cv_pred,nn_cv_pred])

params = {'max_depth':2,
         'eta':0.1,
         'min_child_weight':5,
         'colsample_bytree':1,
         'subsample':1,
         'silent':1,
         'objective': "binary:logistic",
         'eval_metric': 'auc',
         'nthread':8}

stacked_cv_pred=cv(xgb_fit_predict,params,X_stacked,y)

### Load NetMHCpan predictions

In [ ]:
def all_from_fname(fname):
    """Get allele from the filename"""
    # a bit convoluted naming
    return '_'.join(os.path.basename(fname).split('rnd.')[-1].split('.')[0].split('-')[:-1])

def load_allele_table(fname,allele):
    """Load a result table and annotate with allele"""
    #note they are .xls but csv files...
    tmp_table=pd.read_csv(fname,sep='\t')
    tmp_table['allele']=allele
    tmp_table['len']=[len(x) for x in tmp_table.sequence]
    tmp_table['binding']=(tmp_table.meas <= np.log10(500)).astype('int')
    return tmp_table

def load_all_tables(path):
    """Load all tables in directory and concat them into one big table"""
    return pd.concat([load_allele_table(f,all_from_fname(f)) for f in glob.glob(path)])

In [ ]:
#load cv tables
netmhc_cv_res=load_all_tables('benchmark_mhci_reliability/predictions/cv_rnd/netmhcpan/*')

### Evaluate all models

In [ ]:
print "NN AUC:",roc_auc_score(y,nn_cv_pred)
print "XGB AUC:",roc_auc_score(y,xgb_cv_pred)
print "ENSEMBLE AUC:",roc_auc_score(y,stacked_cv_pred)
print "NetMHCpan AUC:", roc_auc_score(netmhc_cv_res.binding,-netmhc_cv_res.pred)